<a href="https://colab.research.google.com/github/MohamedElsayed002/DeepLearning_Study/blob/master/ConvolutionalNeuralNetwork14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Model APIs in Keras


## __Table of Contents__


<ol>
    <li><a href="#Objectives">Objectives</a></li>
    <li>
        <a href="#Setup">Setup</a>
        <ol>
            <li><a href="#Installing-Required-Libraries">Installing Required Libraries</a></li>
            <li><a href="#Importing-Required-Libraries">Importing Required Libraries</a></li>
            <li><a href="#Defining-Helper-Functions">Defining Helper Functions</a></li>
        </ol>
    </li>
    <li><a href="#Types of Model APIs in Keras">Types of Model APIs in Keras</a>
        <ol>
            <li><a href="#Task Definition">Task Definition</a></li>
            <li><a href="#The Sequential Model API">The Sequential Model API</a></li>
            <li><a href="#The Functional Model API">The Functional Model API</a></li>
            <li><a href="#Model Subclassing">Model Subclassing</a></li>
        </ol>
    </li>


## Objectives

After completing this lab you will be able to:

- __Understand__ different use cases for the Sequential and Functional APIs
- __Build__ custom models using sub-classing in Keras

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
print(tf.__version__)
from sklearn import preprocessing
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense
# from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
from sklearn import metrics
from sklearn.model_selection import train_test_split
seed = 7
np.random.seed(seed)

2.15.0


## Types of Model APIs in Keras

There are three main ways of creating models in Keras

* **The Sequential Model**: This is a straightforward way of stacking layers but is limited to having a single-input and single-output stack of layers.
* **The Functional API**: This can support arbitrary model architectures, and is more flexible in enabling complex implementations.

* **Model subclassing**: This is a way of implementing models from scratch. This is mainly used for out-of-the-box research use cases.



In the rest of the lab, we will go through all of these ways of creating models, and walk through a use case for implementing and training each of these model architectures in Keras and Tensorflow.

# Task Definition


In this lab, we will be performing a simple classification task using the 'Sonar' dataset from [UCI](http://archive.ics.uci.edu/ml/datasets/connectionist+bench+\(sonar,+mines+vs.+rocks\)?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML311Coursera747-2022-01-01).


> The label associated with each record contains the letter "R" if the object is a rock and "M" if it is a mine (metal cylinder). The numbers in the labels are in increasing order of aspect angle, but they do not encode the angle directly.



In [ ]:
dataframe = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/labs/Module1/L1/data/sonar.csv", header=None)

dataset = dataframe.values

X = dataset[:, 0:60].astype(float)
y = dataset[:, 60]

# encode labels
le = LabelEncoder()
encoded_y = le.fit(y).transform(y)

Now, we will split the dataset into a training and testing set with a, 80:20 ratio.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, encoded_y, test_size=0.20, random_state=42)

# The Sequential Model API

The Sequential model API groups a linear stack of layers into a `tf.keras.Model`. It doesn't allow you to create models that share layers, have branches, multiple inputs, or multiple outputs. It does provide training and inference features on this model.  

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/labs/Module1/L1/images/sequential_model.png" alt="The Sequential model API" width="50%">


As you'll see,using Sequential API is simple and straightforwared but is only appropriate for simple, more straightforwared tasks, Later in this notebook you'll spend some time building with a more flexible, powerful alternative: The functional API.

# Create the Sequential Model

We start off by initializing the model as an instance of the Sequential class, like so: `model = Sequential()`. We follow this by adding layers, normalization layers, methods, softmax classifiers, and so on to the class, one at a time, using `model.add()`. This can be used to add different types of layer instances on top of the layer stack. Similarly, we can use `model.pop()` to remove the last layer of the model.

Typically, the first layer receives an `input_shape` argument. This is because, in Keras, the shape of the weights is based on the shape of the inputs. The weights are only created when the model first sees some input data. After that, we can omit this argument and automate the inference of the input shape.

Note that if your model is non-linear or contains layers with multiple inputs or outputs, a Sequential model wouldn't be the right choice.

In [ ]:
def baseline_model():
    model = Sequential()
    model.add(Dense(60, activation='relu', input_shape=(60,)))
    model.add(Dense(60, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

    return model


In [ ]:
estimator = baseline_model()
estimator.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 60)                3660      
                                                                 
 dense_4 (Dense)             (None, 60)                3660      
                                                                 
 dense_5 (Dense)             (None, 1)                 61        
                                                                 
Total params: 7381 (28.83 KB)
Trainable params: 7381 (28.83 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Train and Evaluate the Model

After creating the model, compling it with you choice of optimizer and loss function, and doing a sanity check on its contents, you are now ready to build

Simply call the `.fit()` to train the model

In [ ]:
estimator.fit(X_train, y_train, epochs=10, batch_size=16)

Epoch 1/10
11/11 [==============================] - 1s 3ms/step - loss: 0.6850 - accuracy: 0.5361
Epoch 2/10
11/11 [==============================] - 0s 3ms/step - loss: 0.6509 - accuracy: 0.6325
Epoch 3/10
11/11 [==============================] - 0s 3ms/step - loss: 0.6259 - accuracy: 0.7590
Epoch 4/10
11/11 [==============================] - 0s 3ms/step - loss: 0.6058 - accuracy: 0.7530
Epoch 5/10
11/11 [==============================] - 0s 3ms/step - loss: 0.5862 - accuracy: 0.7229
Epoch 6/10
11/11 [==============================] - 0s 3ms/step - loss: 0.5660 - accuracy: 0.7651
Epoch 7/10
11/11 [==============================] - 0s 3ms/step - loss: 0.5491 - accuracy: 0.7771
Epoch 8/10
11/11 [==============================] - 0s 3ms/step - loss: 0.5380 - accuracy: 0.7349
Epoch 9/10
11/11 [==============================] - 0s 3ms/step - loss: 0.5093 - accuracy: 0.7892
Epoch 10/10
11/11 [==============================] - 0s 4ms/step - loss: 0.5024 - accuracy: 0.7470


After that completes, just use `.predict()` to evaluate against your test set. In this case, the `accuracy` will be used as the metric

In [ ]:
y_pred = estimator.predict(X_test)
y_pred = [1 if x >= 0.5 else 0 for x in y_pred]
metrics.accuracy_score(y_pred, y_test)

2/2 [==============================] - 0s 9ms/step


0.8333333333333334

# The Functional Model API

The Sequential class in Keras is typically used for implementing basic neural network architectures. The `Functional` class is typically preferred by most deep learning practitioners as it is more flexible and enables more complex implementations. It allows you to have non-linear topologies, different layers, multiple inputs and outputs. You can easily define branches in your architectures and easily share layers inside architectures. A Functional model is a graph where the nodes of the layers can connect in many more ways than one.

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/labs/Module1/L1/images/functional_model.png" alt="The Sequential model API" width="50%">


All models that are implemented in `Sequential()` can easily be implemented in `Functional()`. Some examples of models that are implemented in `Functional()` include ResNet, GoogleNet, Xception.

Both `Sequential()` and `Functioanl()` use the same training, evaluation, and inference functions

The model class offers a build in trainig loop through the `.fit()` method, and a build-in evaluation loop using the `.evaluate()` method.

The following is a simple example of fitting and evaluating a model on MNIST test data:

We will start by loading the dataset directly using Keras


In [ ]:
def functional_model():
    inputs = keras.Input(shape=(60,))
    layer1 = Dense(60, activation='relu')(inputs)
    layer2 = Dense(60, activation='relu')(layer1)
    outputs = Dense(1, activation='sigmoid')(layer2)

    model = keras.Model(inputs, outputs)

    # Compile model, write code below
    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

    return model

In [ ]:
functional_estimator=functional_model()
estimator.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 60)                3660      
                                                                 
 dense_4 (Dense)             (None, 60)                3660      
                                                                 
 dense_5 (Dense)             (None, 1)                 61        
                                                                 
Total params: 7381 (28.83 KB)
Trainable params: 7381 (28.83 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
functional_estimator.fit(X_train, y_train, epochs=10, batch_size=16)

Epoch 1/10
11/11 [==============================] - 2s 6ms/step - loss: 0.6890 - accuracy: 0.5301
Epoch 2/10
11/11 [==============================] - 0s 5ms/step - loss: 0.6706 - accuracy: 0.5964
Epoch 3/10
11/11 [==============================] - 0s 7ms/step - loss: 0.6622 - accuracy: 0.6446
Epoch 4/10
11/11 [==============================] - 0s 8ms/step - loss: 0.6418 - accuracy: 0.6506
Epoch 5/10
11/11 [==============================] - 0s 5ms/step - loss: 0.6291 - accuracy: 0.6627
Epoch 6/10
11/11 [==============================] - 0s 8ms/step - loss: 0.6121 - accuracy: 0.7108
Epoch 7/10
11/11 [==============================] - 0s 5ms/step - loss: 0.5942 - accuracy: 0.7229
Epoch 8/10
11/11 [==============================] - 0s 11ms/step - loss: 0.5770 - accuracy: 0.7470
Epoch 9/10
11/11 [==============================] - 0s 5ms/step - loss: 0.5512 - accuracy: 0.7831
Epoch 10/10
11/11 [==============================] - 0s 8ms/step - loss: 0.5415 - accuracy: 0.7169


In [ ]:
y_pred = functional_estimator.predict(X_test)
y_pred = [1 if x >= 0.5 else 0 for x in y_pred]
metrics.accuracy_score(y_pred, y_test)

2/2 [==============================] - 0s 6ms/step


0.8571428571428571

# Model Subclassing

Next, we will learn how to make new models and classes using model sub-classing this method is more flexible and can be used to implement out-of-box models, but this comes at a cost it is a lot harder to utilize than `Sequential()` and `Functional()` API

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/labs/Module1/L1/images/subclassing.png" alt="The Sequential model API" width="30%">


We will implement a custom model that performs a type of convolution or pooling. In the following example, we will implement a Sequential model but define it with multiple inputs and outputs

In [ ]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense1 = Dense(60, activation = 'relu')
        self.dense2 = Dense(60, activation = 'relu')
        self.dense3 = Dense(1, activation = 'sigmoid')

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        return self.dense3(x)

def subclass_model():
    inputs = keras.Input(shape=(60,))
    mymodel = MyModel()
    outputs = mymodel.call(inputs)

    model = keras.Model(inputs, outputs)
    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

    return model

In [ ]:
subclass_estimator = subclass_model()
subclass_estimator.fit(X_train, y_train, epochs=15, batch_size=16)


Epoch 1/15
11/11 [==============================] - 4s 8ms/step - loss: 0.7026 - accuracy: 0.5301
Epoch 2/15
11/11 [==============================] - 0s 6ms/step - loss: 0.6745 - accuracy: 0.6145
Epoch 3/15
11/11 [==============================] - 0s 8ms/step - loss: 0.6601 - accuracy: 0.7108
Epoch 4/15
11/11 [==============================] - 0s 10ms/step - loss: 0.6372 - accuracy: 0.7651
Epoch 5/15
11/11 [==============================] - 0s 10ms/step - loss: 0.6224 - accuracy: 0.6867
Epoch 6/15
11/11 [==============================] - 0s 9ms/step - loss: 0.5911 - accuracy: 0.7651
Epoch 7/15
11/11 [==============================] - 0s 8ms/step - loss: 0.5694 - accuracy: 0.7651
Epoch 8/15
11/11 [==============================] - 0s 8ms/step - loss: 0.5428 - accuracy: 0.7590
Epoch 9/15
11/11 [==============================] - 0s 8ms/step - loss: 0.5128 - accuracy: 0.7892
Epoch 10/15
11/11 [==============================] - 0s 8ms/step - loss: 0.4900 - accuracy: 0.8012
Epoch 11/15
11/11

In [ ]:
y_pred = subclass_estimator.predict(X)
y_pred = [1 if x >= 0.5 else 0 for x in y_pred]
metrics.accuracy_score(y_pred, encoded_y)

7/7 [==============================] - 0s 3ms/step


0.8557692307692307